In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/labtest-dataset/messy_cancer_lab_dataset.csv


# Load data

In [2]:
import pandas as pd
df= pd.read_csv('/kaggle/input/labtest-dataset/messy_cancer_lab_dataset.csv')
df.head(10)

order_id signout_date lab_number assigned_to_pathologist      patient_name  \
0      NaN   2021-04-02   LAB81482               Dr. Smith     William Evans   
1  ORD6574   03/16/2021   LAB57052             Dr Jane Doe               NaN   
2  ORD4814   02/17/2022   LAB31319            Justin Smith         Lori Diaz   
3  ORD7572    08-Mar-21   LAB95909               Dr. Smyth     Juan Carrillo   
4  ORD2796    23-Nov-20   LAB60019            Dr. John Lee      Joanna Tyler   
5  ORD3927    05-Aug-20   LAB89818            Dr. Jane Doe     Joseph Murphy   
6  ORD9727   09/13/2023   LAB79163            Dr. Jane Doe  Alexandria Davis   
7  ORD1964   2025-01-28   LAB41195                     NaN       Brandon Cox   
8  ORD2790    24-Jan-21   LAB46509               Dr. Smith   Brandon Aguirre   
9  ORD2796   05/26/2024   LAB82845            Dr. Jane Doe       Gary Cuevas   

              facility         test_category           test   sub_category  \
0  St. Mary's Oncology       Immunohistochem        unknown       Leukemia   
1                  NaN       Immunohistochem  ER/PR Testing    Lung Cancer   
2     General Hospital               Unknown        unknown       Leukemia   
3            St. Marys           Haematology        unknown        Lung CA   
4  Mercy Cancer Center              Cytology        unknown  Not Specified   
5  Mercy Cancer Center               Unknown   HER2 Testing    Lung Cancer   
6  St. Mary's Oncology             histology        unknown  breast cancer   
7                  NaN  Immunohistochemistry  ER/PR Testing       Leukemia   
8     General Hospital             Histology        unknown  Breast Cancer   
9  Mercy Cancer Center               Unknown   HER2 Testing  breast cancer   

  receiving_centers processing_centers creation_date  service     price  \
0               NaN                NaN    2022-05-09  routine       206   
1           Unknown           Main Lab    06/20/2023  Consult  ksh 3833   
2             Lab B        central lab     10-Mar-25  Unknown      1016   
3          Main Lab                NaN     12-Jan-25  routine      2366   
4               NaN           Main Lab    2022-06-09  routine     ksh 0   
5          Main Lab           Main Lab     08-May-21  Routine       NaN   
6             lab a            Unknown    2021-05-22  Routine      3870   
7             Lab A        Central Lab    12/26/2021  routine      1712   
8             Lab A              Lab C    09/21/2022   Urgent      2797   
9             Lab A        Central Lab     26-Oct-22   Urgent     ksh 0   

  payment_method  
0            NaN  
1            NaN  
2           Cash  
3         credit  
4        Unknown  
5            NaN  
6         credit  
7      Insurance  
8            INS  
9      Insurance

In [3]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   order_id                 9484 non-null   object
 1   signout_date             9506 non-null   object
 2   lab_number               9502 non-null   object
 3   assigned_to_pathologist  8570 non-null   object
 4   patient_name             9451 non-null   object
 5   facility                 8736 non-null   object
 6   test_category            10000 non-null  object
 7   test                     9500 non-null   object
 8   sub_category             8911 non-null   object
 9   receiving_centers        8281 non-null   object
 10  processing_centers       8390 non-null   object
 11  creation_date            9756 non-null   object
 12  service                  8499 non-null   object
 13  price                    9303 non-null   object
 14  payment_method           8603 non-null 

order_id signout_date lab_number assigned_to_pathologist  \
count      9484         9506       9502                    8570   
unique     5570         4657       8203                    2818   
top     ORD6339   2023/13/01   LAB70910               Dr. Smyth   
freq          9          247          6                    1013   

         patient_name facility         test_category     test sub_category  \
count            9451     8736                 10000     9500         8911   
unique           8876        7                    10        9            8   
top     Michael Brown  Unknown  Immunohistochemistry  unknown      Lung CA   
freq                6     1300                  1043     3135         1145   

       receiving_centers processing_centers creation_date  service  price  \
count               8281               8390          9756     8499   9303   
unique                 5                  5          4741        6   6763   
top                lab a           Main Lab       invalid  Consult  ksh 0   
freq                1689               1721           143     1464    321   

       payment_method  
count            8603  
unique              6  
top           Unknown  
freq             1469

# Task 1: Data Cleaning & Standardization

# Data Cleaning Summary
The cleaning process focused on transforming the messy cancer diagnostics dataset into a structured format suitable for analysis and modeling. Key steps included:

Handling Missing Values: Filled missing identifiers (e.g., order_id, lab_number) with unique synthetic IDs and replaced other null fields with 'Unknown' to preserve row integrity.

Standardizing Dates: Unified creation_date and signout_date formats into YYYY-MM-DD using flexible parsing, while flagging unrecognizable formats as 'Invalid'.

Cleaning Price Field: Removed currency prefixes, stripped whitespace, and converted to consistent numeric values for modeling.

Normalizing Categorical Fields: Standardized text capitalization and corrected typos and semantic inconsistencies across columns like facility, test_category, assigned_to_pathologist, and others.

Saving Outputs: Exported the cleaned dataset for downstream ETL, database loading, and machine learning tasks. **

In [8]:
import pandas as pd
import re
from datetime import datetime
from dateutil import parser  # Added for more flexible date parsing

# Load the dataset
df = pd.read_csv('/kaggle/input/labtest-dataset/messy_cancer_lab_dataset.csv')

# 1. Handle Missing Values
df['order_id'] = df['order_id'].fillna('ORD' + pd.Series(range(1, len(df) + 1)).astype(str).str.zfill(4))
df['lab_number'] = df['lab_number'].fillna('LAB' + pd.Series(range(1, len(df) + 1)).astype(str).str.zfill(5))

fill_unknown_cols = [
    'assigned_to_pathologist', 'patient_name', 'facility', 
    'receiving_centers', 'processing_centers', 
    'test', 'sub_category', 'service', 'payment_method'
]
for col in fill_unknown_cols:
    df[col] = df[col].fillna('Unknown')

df['creation_date'] = df['creation_date'].fillna('Unknown')
df['signout_date'] = df['signout_date'].fillna('Unknown')

df['price'] = df['price'].fillna('ksh 0')

# 2. Improved Date Standardization
def standardize_date(date_str):
    # Handle invalid or missing inputs
    if pd.isnull(date_str) or str(date_str).lower().strip() in ['unknown', 'invalid', 'nan', '']:
        return 'Unknown'
    
    # Clean the date string
    date_str = str(date_str).strip()
    # Remove extra spaces, punctuation, or common typos
    date_str = re.sub(r'\s+', ' ', date_str)  # Replace multiple spaces with single
    date_str = re.sub(r'[^\w\s/-]', '', date_str)  # Remove unexpected characters except alphanumeric, space, /, -

    # Define common date formats
    date_formats = [
        '%Y-%m-%d', '%d/%m/%Y', '%m/%d/%Y', '%Y/%m/%d',
        '%d-%m-%Y', '%m-%d-%Y', '%Y.%m.%d', '%d.%m.%Y',
        '%b %d %Y', '%d %b %Y', '%Y %b %d', '%B %d %Y', '%d %B %Y'
    ]
    
    # Try parsing with specific formats
    for fmt in date_formats:
        try:
            parsed_date = datetime.strptime(date_str, fmt)
            return parsed_date.strftime('%Y-%m-%d')
        except ValueError:
            continue
    
    # Fallback to dateutil.parser for flexible parsing
    try:
        parsed_date = parser.parse(date_str, dayfirst=True, fuzzy=True)
        return parsed_date.strftime('%Y-%m-%d')
    except (ValueError, TypeError):
        # Log invalid date for debugging
        with open('invalid_dates.log', 'a') as f:
            f.write(f"Invalid date: {date_str}\n")
        return 'Invalid'

# Apply date standardization
df['creation_date'] = df['creation_date'].apply(standardize_date)
df['signout_date'] = df['signout_date'].apply(standardize_date)

# 3. Price Cleanup
df['price'] = df['price'].astype(str).str.replace('ksh', '', regex=False).str.strip()
df['price'] = pd.to_numeric(df['price'], errors='coerce').fillna(0).astype(int)

# 4. Standardize Categorical Fields
df['payment_method'] = df['payment_method'].str.lower().replace({
    'ins': 'insurance', 'credit': 'credit', 'cash': 'cash'
}).str.capitalize()

df['assigned_to_pathologist'] = df['assigned_to_pathologist'].str.title().replace({
    'Dr. Smyth': 'Dr. Smith',
    'Dr Jane Doe': 'Dr. Jane Doe'
})

df['test_category'] = df['test_category'].str.lower().replace({
    'immunohistochem': 'immunohistochemistry',
    'haematology': 'hematology'
}).str.capitalize()

df['test'] = df['test'].str.title().replace({
    'Er/Pr Testing': 'ER/PR Testing',
    'Her2 Testing': 'HER2 Testing'
})

df['sub_category'] = df['sub_category'].str.lower().replace({
    'lung ca': 'lung cancer',
    'breast ca': 'breast cancer',
    'not specified': 'unknown'
}).str.capitalize()

df['facility'] = df['facility'].str.title().replace({
    'St. Marys': "St. Mary's Oncology"
})

df['service'] = df['service'].str.lower().str.capitalize()

df['receiving_centers'] = df['receiving_centers'].str.lower().replace({
    'lab a': 'Lab A', 'lab b': 'Lab B', 'main lab': 'Main Lab'
}).str.title()

df['processing_centers'] = df['processing_centers'].str.lower().replace({
    'lab a': 'Lab A', 'lab b': 'Lab B', 'main lab': 'Main Lab',
    'central lab': 'Central Lab', 'lab c': 'Lab C'
}).str.title()

# 5. Save cleaned dataset
df.to_csv('cleaned_dataset.csv', index=False)

# 6. Final Summary
print("Cleaning completed. Summary:")
print(f"Total records: {len(df)}")
print(f"Invalid creation_dates: {len(df[df['creation_date'] == 'Invalid'])}")
print(f"Invalid signout_dates: {len(df[df['signout_date'] == 'Invalid'])}")

Cleaning completed. Summary:
Total records: 10000
Invalid creation_dates: 0
Invalid signout_dates: 0


# Task 2: ETL Pipeline  

## Structured Output – SQLite Database

As part of the ETL pipeline, the cleaned lab test data was loaded into a structured SQLite database named `cleaned_labtest.db`. This enables efficient querying, storage, and downstream analysis.

###  Table: `labtest`
- Contains 15 columns including identifiers, dates, test details, facility info, and payment metadata.
- Dates were standardized and invalid entries flagged.
- Price field converted to numeric format.
- All categorical values were normalized for consistency.

###  Sample Queries
- Count total rows: `SELECT COUNT(*) FROM labtest;`
- Average price per test category
- Delay prediction prep via date fields (`creation_date`, `signout_date`)

This database is suitable for integration into dashboards, machine learning workflows, or hospital analytics platforms.
******

In [9]:
def extract_data(file_path):
    import pandas as pd
    return pd.read_csv('cleaned_dataset.csv')


**Transform**

In [10]:
def transform_data(df):
    # Call all your cleaning steps here as modular functions:
    df = handle_missing_values(df)
    df = standardize_dates(df)
    df = clean_price(df)
    df = fix_typos(df)
    return df


**Load**

In [11]:
import sqlite3

def load_data_to_sqlite(df, db_path='/kaggle/working/cleaned_labtest.db', table_name='labtest'):
    conn = sqlite3.connect(db_path)
    df.to_sql(table_name, conn, index=False, if_exists='replace')
    conn.close()


In [12]:
load_data_to_sqlite(df, '/kaggle/working/cleaned_labtest.db')

In [13]:
import os
print(os.listdir('/kaggle/working'))


['cleaned_labtest.db', 'cleaned_dataset.csv', '.virtual_documents']


**Querying Your SQLite DB**

In [15]:
import sqlite3
import pandas as pd

# Connect to the database
conn = sqlite3.connect('/kaggle/working/cleaned_labtest.db')

# List all tables to confirm it's there
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
print(" Tables in database:")
print(tables)

# Query first few rows from 'labtest' table
sample_data = pd.read_sql_query("SELECT * FROM labtest LIMIT 5;", conn)
print("\n Sample rows:")
print(sample_data)

# Optional: Check column names and types
metadata = pd.read_sql_query("PRAGMA table_info(labtest);", conn)
print("\n Table schema:")
print(metadata)

conn.close()


 Tables in database:
      name
0  labtest

 Sample rows:
  order_id signout_date lab_number assigned_to_pathologist   patient_name  \
0  ORD0001   2021-04-02   LAB81482               Dr. Smith  William Evans   
1  ORD6574   2021-03-16   LAB57052            Dr. Jane Doe        Unknown   
2  ORD4814   2022-02-17   LAB31319            Justin Smith      Lori Diaz   
3  ORD7572   2021-03-08   LAB95909               Dr. Smith  Juan Carrillo   
4  ORD2796   2020-11-23   LAB60019            Dr. John Lee   Joanna Tyler   

              facility         test_category           test sub_category  \
0  St. Mary'S Oncology  Immunohistochemistry        Unknown     Leukemia   
1              Unknown  Immunohistochemistry  ER/PR Testing  Lung cancer   
2     General Hospital               Unknown        Unknown     Leukemia   
3  St. Mary's Oncology            Hematology        Unknown  Lung cancer   
4  Mercy Cancer Center              Cytology        Unknown      Unknown   

  receiving_centers pr

**simple sql querry**

In [16]:

import sqlite3

# Reconnect to your saved DB file in Kaggle
conn = sqlite3.connect('/kaggle/working/cleaned_labtest.db')

# Total number of records
pd.read_sql_query("SELECT COUNT(*) FROM labtest;", conn)

# Unique facilities
pd.read_sql_query("SELECT DISTINCT facility FROM labtest;", conn)

# Average price per test category
pd.read_sql_query("""
    SELECT test_category, AVG(price) as avg_price
    FROM labtest
    GROUP BY test_category
    ORDER BY avg_price DESC;
""", conn)


test_category    avg_price
0  Immunohistochemistry  4534.396396
1               Unknown  4534.027163
2              Cytology  4530.078652
3             Histology  4524.887846
4            Hematology  4462.169574
5                   Ihc  4450.762245

In [17]:
conn.close()
print("\nDatabase connection closed.")


Database connection closed.


# Task 3: Machine Learning task 

## Machine Learning Task – Predicting Lab Test Delays

The selected use case involves predicting the number of days between a test’s creation and its signout date. This delay metric helps identify operational bottlenecks across labs and facilities.

###  Approach Summary
- **Target Variable:** `delay_days` calculated as the difference between standardized `creation_date` and `signout_date`
- **Preprocessing:** Invalid or missing dates flagged; outliers (e.g. negative delays) excluded
- **Feature Engineering:**
  - Categorical encoding for fields like `facility`, `test_category`, `service`, and `assigned_to_pathologist`
  - Date-derived features: month, day of week, weekend indicator
  - Price normalization and inclusion of logistical variables (e.g. receiving/processing centers)
- **Modeling Strategy:** Regression models (e.g., Linear Regression, Random Forest,) with evaluation using MAE and RMSE

This predictive task supports workload forecasting and turnaround optimization in real-world diagnostic labs.
